In [9]:
import os
project_path = "D:/workspace/Bus Project"
os.chdir(project_path)

In [10]:
from multiprocessing import Pool
import multiprocessing
import pandas as pd
import numpy as np
import datetime

import bus.analyzer as anz

In [11]:
def parallelize_dataframe(func, df, core = multiprocessing.cpu_count()-4):
    df_split = np.array_split(df, core)
    pool = Pool(core)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [12]:
# 데이터 기간 설정
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)

# 로딩할 파일 명 리스트 생성
input_path_list = anz.make_input_path(start_date, end_date)

usage_df = parallelize_dataframe(anz.load_total_usage_data, input_path_list, core = 10)

cluster_df = anz.load_cluster_df()

In [13]:
cluster_df

,station_id,cluster_id,cluster_group,cluster_target,cluster_longitude,cluster_latitude
0,638.0,0,제주시,0,126.486426,33.501894
1,149.0,1,제주시,1,126.553375,33.478634
2,1355.0,0,제주시,0,126.486426,33.501894
3,1564.0,1,제주시,1,126.553375,33.478634
4,150.0,1,제주시,1,126.553375,33.478634
...,...,...,...,...,...,...
3649,1888.0,1486,서귀포시,637,126.607060,33.290130
3650,2212.0,1487,서귀포시,638,126.882770,33.442380
3651,2208.0,1488,서귀포시,639,126.845280,33.374930
3652,3472.0,1489,서귀포시,640,126.759700,33.357750


In [30]:
usage_selector = ['user_id', 'geton_datetime', 'geton_station_id', 'getoff_datetime', 'getoff_station_id']
usage_df = usage_df[usage_selector]

cluster_selector = ['station_id', 'cluster_group', 'cluster_target']
cluster_df = cluster_df[cluster_selector]

geton_columns = {}
getoff_columns = {}
for column in cluster_selector:
    geton_columns[column] = "geton_"+column
    getoff_columns[column] = "getoff_"+column
geton_cluster_df = cluster_df.rename(columns=geton_columns)
getoff_cluster_df = cluster_df.rename(columns=getoff_columns)

merged_df = usage_df
merged_df = pd.merge(merged_df, geton_cluster_df, on=['geton_station_id'])
# merged_df = pd.merge(merged_df, getoff_cluster_df, on=['getoff_station_id'])
# merged_selector = ['user_id', 'geton_datetime', 'geton_cluster_group', 'geton_cluster_target', 'getoff_datetime', 'getoff_station_id']
# merged_df = merged_df[merged_selector]

In [32]:
merged_df

,user_id,geton_datetime,geton_station_id,getoff_datetime,getoff_station_id,geton_cluster_group,geton_cluster_target
0,f6f372cf8c6732eafc2a82b4f9d7a08bb3b493213ea4ef...,2019-06-01 15:02:25,988,2019-06-01 15:16:38,26.0,제주시,49
1,3f6cfe2429f6e8d62cddc39222993405789b7b9ea2a592...,2019-06-01 15:02:29,988,2019-06-01 15:16:40,26.0,제주시,49
2,83219e75a6821a91e6d69f354b2bd0814c7f511fd3b5f6...,2019-06-01 10:34:23,988,2019-06-01 11:06:40,360.0,제주시,49
3,6f57930abc4e2d5a4fba78603e51af6d14dd84283cafc5...,2019-06-01 13:46:52,988,2019-06-01 14:50:06,2969.0,제주시,49
4,99289d8e7b3ffbd16e5184d6709b8bdf2c18a8984b4ff1...,2019-06-01 16:34:01,988,2019-06-01 16:34:07,988.0,제주시,49
...,...,...,...,...,...,...,...
12290841,e26d92cf1d3b26ea1b400baca37f55460f7dfd33a068bf...,2019-08-20 16:49:21,3559,2019-08-20 17:54:16,1228.0,제주시,609
12290842,68a8566bfb0b6b6256d2c8b71ac0f4aab81f8932262d1f...,2019-08-20 16:49:24,3559,2019-08-20 17:54:15,1228.0,제주시,609
12290843,930ef3d55183f4d7a69e2bbfcf9405b627ce5abc731c9f...,2019-08-27 14:24:41,3559,NaT,NaN,제주시,609
12290844,db6b65142fa12acf4b38dae2ed7f6ca8a14e981a334b5b...,2019-08-28 20:14:27,3559,NaT,NaN,제주시,609


In [21]:
usage_df

,user_id,geton_datetime,geton_station_id,getoff_datetime,getoff_station_id
0,f6f372cf8c6732eafc2a82b4f9d7a08bb3b493213ea4ef...,2019-06-01 15:02:25,988,2019-06-01 15:16:38,26.0
1,f6f372cf8c6732eafc2a82b4f9d7a08bb3b493213ea4ef...,2019-06-01 15:25:03,26,2019-06-01 16:02:07,389.0
2,3f6cfe2429f6e8d62cddc39222993405789b7b9ea2a592...,2019-06-01 15:02:29,988,2019-06-01 15:16:40,26.0
3,3f6cfe2429f6e8d62cddc39222993405789b7b9ea2a592...,2019-06-01 20:13:06,3271,2019-06-01 20:45:40,187.0
4,3f6cfe2429f6e8d62cddc39222993405789b7b9ea2a592...,2019-06-01 16:45:21,342,NaT,NaN
...,...,...,...,...,...
942059,02b1d9245a36332114146cdbd7acf307d56eaf2a79dbae...,2019-08-29 00:18:19,150,2019-08-29 00:40:43,475.0
942060,5349769af16ed3978d92ca42931c53d74ac506a2d3319f...,2019-08-29 00:05:43,305,2019-08-29 00:24:22,479.0
942061,9b6e974566387c96cc25780717eb50408e64610a5c7934...,2019-08-29 00:10:20,1355,2019-08-29 00:16:43,4033.0
942062,90a4d8db628d6c9796682aaf19c4f500f7c34699b42a00...,2019-08-29 00:10:18,1355,2019-08-29 00:16:38,4033.0


In [36]:
station_df = anz.load_cluster_df()
station_df

,station_id,cluster_id,cluster_group,cluster_target,cluster_longitude,cluster_latitude
0,638.0,0,제주시,0,126.486426,33.501894
1,149.0,1,제주시,1,126.553375,33.478634
2,1355.0,0,제주시,0,126.486426,33.501894
3,1564.0,1,제주시,1,126.553375,33.478634
4,150.0,1,제주시,1,126.553375,33.478634
...,...,...,...,...,...,...
3649,1888.0,1486,서귀포시,637,126.607060,33.290130
3650,2212.0,1487,서귀포시,638,126.882770,33.442380
3651,2208.0,1488,서귀포시,639,126.845280,33.374930
3652,3472.0,1489,서귀포시,640,126.759700,33.357750
